$\def \dot #1#2{\left\langle #1, #2 \right\rangle}$
$\def \adot #1#2{\left\langle #1, #2 \right\rangle}$
$\def \cD {\mathcal{D}}$
$\def \cW {\mathcal{W}}$
$\def \bc {\mathbf{c}}$
$\def \bv {\mathbf{v}}$
$\def \bG {\mathbf{G}}$
$\def \bC {\mathbf{C}}$
$\def \bU {\mathbf{U}}$
$\def \bV {\mathbf{V}}$
$\def \bW {\mathbf{W}}$
$\def \bPhi {\mathbf{\Phi}}$
$\def \bPsi {\mathbf{\Psi}}$
$\def \bGamma {\mathbf{\Gamma}}$
$\def \bSigma {\mathbf{\Sigma}}$
$\def \bOmega {\mathbf{\Omega}}$
$\def \bbE {\mathbb{E}}$
$\def \bbP {\mathbb{P}}$
$\def \bbR {\mathbb{R}}$
$\def \bbN {\mathbb{N}}$

# Demonstrating the proper PCA decomposition

From a snapshot set $\{ u_i \}_{i=1}^N$, how do we derive the proper PCA fit, noting that the covariance is properly defined as
$$
\langle v, C w \rangle := \mathbb{E}(\langle u, v \rangle \langle u, w \rangle) 
$$
but here we use the approximate (empirical) covariance:
$$
\langle v, C w \rangle = \frac{1}{N} \sum_{i=1}^N \langle u_i, v \rangle \langle u_i, w \rangle
$$

What I actually did was the eigen-decomposition of the Gram matrix $\mathbf{G}$, where $G_{i,j} = \langle u_i, u_j \rangle$, and used this to build the "PCA", but apparently that wasn't right... Well, lets see. 

Note firstly that $C: V \to V$ and $\bG : \mathbb{R}^N \to \mathbb{R}^N$. Let's take the case $V=\mathbb{R}^K$ to  make things a bit simpler, then we have $\bC: \bbR^K \to \bbR^K$ and our empirical covariance is
$$
\langle v, \bC w \rangle = v^T \bC w = \frac{1}{N} \sum_{i=1}^N \langle u_i, v \rangle \langle u_i, w \rangle = \frac{1}{N} \sum_{i=1}^N (u_i^T v)^T ( u_i^T w)
$$
Let us write $\bU = [u_1, u_2, \ldots, u_N] \in \bbR^{K\times N}$, so we have from the above

$$
v^T \bC w = v^T \bU \bU^T w,
$$

so indeed as we wrote on the board the other day we have that $\bC$ is the outer-product matrix of $\bU$. Note in this case also we have the Gram matrix $\bG = \bU^T \bU$. 

There's the SVD decomposition of $\bU = \bPsi \bSigma \bPhi^T$, with $\bPsi\in\bbR^{K\times K}$, $\bSigma\in\bbR^{N\times K}$ and $\bPhi \in \bbR^{N \times N}$. We may have fewer than $N$ singular values. Both $\bC$ and $\bG$ are evidently symmetric matrices and they decompose as

$$
\bG =  \bU \bU^T  = \bPsi \bSigma \bSigma^T \bPsi^T \quad\text{and}\quad \bC = \bU^T \bU = \bPhi \bSigma^T \bSigma \bPhi^T
$$

Note that $\bSigma \bSigma^T$ is a diagonal $K\times K$ matrix, while $\bSigma^T \bSigma$ is $N\times N$, they both are diagonal with $\sigma^2$ along the diagonal.

Now, we have that $\bPsi = \bU \bPhi \bSigma^{-1}$. I'm reasonably sure this all applies if we consider a more general $V$, with of course the addition of an operator $E : v \to \bbR^K$ that maps from a canonical ortho basis to the coordinates. But this doesn't complicate things too much.

First let us test this all in $\bbR^K$ for some moderate $K$.

In [2]:
import numpy as np
import scipy as sp
import importlib
import seaborn as sns
import matplotlib.pyplot as plt
import pdb

import sys
sys.path.append("../../")
import pyApproxTools as pat
importlib.reload(pat)

%matplotlib inline

In [3]:
K = 10
N = 4

# First make a random orthonormal vector
Psi_orig = sp.stats.ortho_group.rvs(dim=K)
sigma = np.sort(np.random.random(K))[::-1]
D_orig = np.diag(sigma**2)

# This is the original covariance matrix!
Cov_orig = Psi_orig * D_orig * Psi_orig.T

points = np.random.multivariate_normal(np.zeros(K), Cov_orig, N)
U = points.T
print('K={0}, N={1}, U is dim {2}'.format(K,N,U.shape))

K=10, N=4, U is dim (10, 4)


In the above code we generate $N$ random points in $\bbR^K$ that are distributed according to a randomly generated "PCA construction", that is a random ortho-basis ```Psi_orig``` and a randomly generated sequence ```sigma``` or ordered numbers between 0 and 1, from which ```Cov_orig``` is calculated in the obvious way, and $U$ are the multi-variate normal random numbers.

### Now we calculate the PCA in two ways. First by factoring $\bU^T \bU$, second by factoring $\bU\bU^T$, but lets make sure we get the same quantities
Recal $\bU \in \bbR^{K\times N}$. We are doing:

$$
\bG =  \bU \bU^T  = \bPsi \bSigma^2 \bPsi^T \quad\text{and}\quad \bC = \bU^T \bU = \bPhi \bSigma^2 \bPhi^T
$$

and as $\bU = \bPsi \bSigma \bPhi^T$ we should be able to recover the first $N$ columns of $\bPsi$ from the calculation 

$$\bPsi = \bU \bPhi \bSigma^{-1}$$ 

Recall $\bPsi\in\bbR^{K\times K}$, $\bSigma\in\bbR^{N\times K}$ and $\bPhi \in \bbR^{N \times N}$

In [13]:
G = U.T @ U
C = U @ U.T

sigma_1, Phi = np.linalg.eigh(G)
sigma_2, Psi = np.linalg.eigh(C)

# Because NumPy outputs eigenvalues in reverse (increasing) order, we reverse
sigma_1 = sigma_1[::-1]
sigma_2 = sigma_2[::-1]
Phi = Phi[:,::-1]
Psi = Psi[:,::-1]

# Embed the singular values diagonally in the appropriate (K x N) matrix
Sigma_inv = np.pad(np.diag(1.0/np.sqrt(sigma_1)), ((0,K-N), (0, 0)), 'constant')

print('Psi (first N columns):\n\n', Psi[:,:N], 
      '\n\nU Phi Sigma_inv (first N columns, rest are 0):\n\n', U @ Phi @ Sigma_inv.T[:,:N])

Psi (first N columns):

 [[-0.02165112  0.24978483  0.47281743 -0.14225278]
 [-0.01752974  0.00949452  0.01979772 -0.14891587]
 [-0.27033968 -0.84239575  0.27523655  0.30770756]
 [-0.82690875  0.0569196  -0.47675846 -0.19785334]
 [ 0.33472656 -0.47069387 -0.2324029  -0.71302639]
 [-0.35829693  0.04943554  0.62922423 -0.44140978]
 [ 0.01041433  0.01762553  0.0124242  -0.34314388]
 [ 0.00485373 -0.01466851  0.01473459  0.01746618]
 [-0.04014061  0.00228673  0.15010597  0.04678117]
 [ 0.01415885  0.01092257 -0.01266612 -0.0163656 ]] 

U Phi Sigma_inv (first N columns, rest are 0):

 [[ 0.02165112 -0.24978483  0.47281743 -0.14225278]
 [ 0.01752974 -0.00949452  0.01979772 -0.14891587]
 [ 0.27033968  0.84239575  0.27523655  0.30770756]
 [ 0.82690875 -0.0569196  -0.47675846 -0.19785334]
 [-0.33472656  0.47069387 -0.2324029  -0.71302639]
 [ 0.35829693 -0.04943554  0.62922423 -0.44140978]
 [-0.01041433 -0.01762553  0.0124242  -0.34314388]
 [-0.00485373  0.01466851  0.01473459  0.01746618]
 [ 0.

In the above results we see that the first $N$ columns of $\bPsi$ is recovered from the calculation of $\bU \bPhi \bSigma^{-1}$, up to a difference of sign. The difference of sign is due to the ambiguity of sign in the SVD decomposition, we can see for example that $ \psi_i \sigma_i \phi_j^T = (-\psi_i) \sigma_i (-\phi_j)^T$. 

My point in showing this is that the $N$-dimensional basis $[\psi_1,\ldots,\psi_N]$ of the best-fit PCA basis can be found purely from the matrix $\bG_{i,j} = \langle u_i - \bar{u}, u_j - \bar{u} \rangle$ (noting above that we've assumed that $\bar{u} = 0$). This is a much smaller $N\times N$ calculation and doesn't require some pre-built orthonormal basis of $V$. Now, the problem is of course extending to the rest of the columns of $\Psi$, so that we can do the calculations of the sub-matrices of $S$ or $T$.

In [5]:
# A few further sanity checks here:
U1,Sig,V1 = np.linalg.svd(U)
Sigma = np.pad(np.diag(np.sqrt(sigma_1)), ((0,K-N), (0, 0)), 'constant')

print('Sigma from G:       ', sigma_1[:5], '...')
print('Sigma from C:       ', sigma_2[:5], '...')
print('Sigma from SVD:     ', Sig*Sig)
print('(Psi.T @ U @ Phi)^2:', np.diag(Psi.T @ U @ Phi)**2)
print('')
print('Psi is dim   ', Psi.shape)
print('Phi is dim   ', Phi.shape)
print('Sigma is dim ', Sigma.shape, '\n')

print('U:                  \n', U @ Phi)
print('Psi Sigma Phi^T:    \n', (Psi) @ Sigma, '\n')

Sigma from G:        [0.88510719 0.34664984 0.30016335 0.04373008] ...
Sigma from C:        [8.85107187e-01 3.46649844e-01 3.00163353e-01 4.37300842e-02
 8.86574883e-18] ...
Sigma from SVD:      [0.88510719 0.34664984 0.30016335 0.04373008]
(Psi.T @ U @ Phi)^2: [0.88510719 0.34664984 0.30016335 0.04373008]

Psi is dim    (10, 10)
Phi is dim    (4, 4)
Sigma is dim  (10, 4) 

U:                  
 [[ 0.02036941 -0.14706575  0.25904327 -0.02974753]
 [ 0.016492   -0.00559009  0.01084661 -0.0311409 ]
 [ 0.25433593  0.49597715  0.15079431  0.064347  ]
 [ 0.77795687 -0.03351254 -0.26120245 -0.04137458]
 [-0.3149112   0.27713033 -0.12732696 -0.14910622]
 [ 0.33708623 -0.02910615  0.34473412 -0.09230646]
 [-0.00979782 -0.01037738  0.00680686 -0.07175735]
 [-0.00456639  0.00863638  0.00807267  0.00365248]
 [ 0.03776434 -0.00134636  0.0822388   0.00978276]
 [-0.01332067 -0.00643088 -0.00693941 -0.00342233]]
Psi Sigma Phi^T:    
 [[-0.02036941  0.14706575  0.25904327 -0.02974753]
 [-0.016492    0.